# mnist_softmax

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 데이터 사진 모양 28*28 = 784
linear = torch.nn.Linear(784, 10, bias=True).to(device)

# 초기화
torch.nn.init.normal(linear.weight)

# cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate)

/var/folders/gt/0rjz50991_387k9xggv9vwww0000gn/T/ipykernel_8637/4079555409.py:23: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(linear.weight)


In [2]:
mnist_train = dsets.MNIST(root="MNIST_data/", train=True, transform=\
                         transforms.ToTensor(), download=True)

mnist_test = dsets.MNIST(root="MNIST_data/", train=False, transform=\
                         transforms.ToTensor(), download=True)

In [3]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [4]:
for epoch in range(training_epochs):
    for X, Y in data_loader:
        X = X.view(-1, 28*28).to(device)

In [5]:
# train
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 5.766133785
Epoch: 0002 cost = 1.576762676
Epoch: 0003 cost = 1.043858647
Epoch: 0004 cost = 0.831811786
Epoch: 0005 cost = 0.713422596
Epoch: 0006 cost = 0.634814620
Epoch: 0007 cost = 0.579115331
Epoch: 0008 cost = 0.536338866
Epoch: 0009 cost = 0.502945781
Epoch: 0010 cost = 0.476410121
Epoch: 0011 cost = 0.454403371
Epoch: 0012 cost = 0.435916156
Epoch: 0013 cost = 0.420033962
Epoch: 0014 cost = 0.406569064
Epoch: 0015 cost = 0.394932747
Learning finished


# mnist_nn

In [6]:
# nn layers
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [7]:
# Initialization
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[ 0.0478, -0.5218, -0.5596,  ...,  0.5776, -1.0125,  1.8102],
        [-0.2905, -3.2769, -0.5354,  ..., -0.0161, -0.2608,  0.2602],
        [ 0.8661, -1.4953,  1.9001,  ...,  1.2469, -0.0206,  0.0991],
        ...,
        [-0.1032,  1.2246,  0.1657,  ...,  0.6953, -0.9528,  1.1839],
        [ 1.8482,  0.3867,  0.1505,  ...,  1.5141,  1.3169, -0.4655],
        [-0.2702, -0.7344,  0.2048,  ...,  1.9289, -0.0597, -0.2786]],
       requires_grad=True)

In [8]:
# model / 마지막에 relu를 하지 않은 이유 : cross 엔트로피를 하기 위해 소프트 맥스 적용
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [9]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 157.207214355
Epoch: 0002 cost = 36.997970581
Epoch: 0003 cost = 23.290493011
Epoch: 0004 cost = 15.998687744
Epoch: 0005 cost = 11.764789581
Epoch: 0006 cost = 8.422083855
Epoch: 0007 cost = 6.234779358
Epoch: 0008 cost = 4.646455765
Epoch: 0009 cost = 3.416964531
Epoch: 0010 cost = 2.626241446
Epoch: 0011 cost = 2.004442930
Epoch: 0012 cost = 1.496607542
Epoch: 0013 cost = 1.143470407
Epoch: 0014 cost = 0.877994061
Epoch: 0015 cost = 0.765159547
Learning finished


In [11]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9488000273704529
Label:  8
Prediction:  5


/Users/gyuyeonjo/opt/anaconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/gyuyeonjo/opt/anaconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
